<a href="https://colab.research.google.com/github/zshuyinggg/RAG_yelp_chatbot/blob/main/toy_example.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!pip install faiss-cpu torch transformers langchain

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.6/50.6 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.5/27.5 MB 44.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 34.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 408.0/408.0 kB 21.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 296.9/296.9 kB 20.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.0/78.0 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 144.5/144.5 kB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 4.8 MB/s eta 0:00:00


In [1]:
import faiss
import numpy as np
import torch
from transformers import AutoTokenizer, AutoModel, GPT2LMHeadModel
from langchain.chains import ConversationChain
# Initialize FAISS index

index = faiss.IndexFlatL2(384)  # Assuming 768-dimensional embeddings


# Load pre-trained models

embedding_model = AutoModel.from_pretrained("sentence-transformers/all-MiniLM-L6-v2")
embedding_model.save_pretrained('/mnt/disk1/models/all-MiniLM-L6-v2_models/')
tokenizer = AutoTokenizer.from_pretrained("sentence-transformers/all-MiniLM-L6-v2")
tokenizer.save_pretrained('/mnt/disk1/models/all-MiniLM-L6-v2_tokenizer/')
generation_model = GPT2LMHeadModel.from_pretrained("gpt2")
generation_model.save_pretrained('/mnt/disk1/models/gpt2_models/')
generation_tokenizer = AutoTokenizer.from_pretrained("gpt2")
generation_tokenizer.save_pretrained('/mnt/disk1/models/gpt2_tokenizer/')



ModuleNotFoundError: No module named 'transformers'

In [8]:
# Sample data for indexing
restaurant_data = [
    {"name": "Tony's Pizza Napoletana", "cuisine": "Italian", "location": "San Francisco", "review": "Great pizzas!"},
    {"name": "Flour + Water", "cuisine": "Italian", "location": "San Francisco", "review": "Delicious handmade pasta."}
]

# Embedding and indexing data
def index_data(data):
    embeddings = []
    for item in data:
        text = f"{item['name']} {item['cuisine']} {item['location']} {item['review']}"
        tokens = tokenizer(text, return_tensors="pt")
        embedding = embedding_model(**tokens).last_hidden_state.mean(dim=1).detach().numpy()
        embeddings.append(embedding)
        index.add(embedding)
    return embeddings
index = faiss.IndexFlatL2(384)  # Assuming 768-dimensional embeddings
index_data(restaurant_data)

[array([[-1.29918113e-01,  3.51073891e-01, -6.32777363e-02,
          4.26706910e-01, -7.15602934e-01,  3.73170853e-01,
          1.82476878e-01, -1.66640490e-01, -7.67187774e-02,
         -6.81655407e-01, -1.84582815e-01, -7.28175938e-02,
          1.90383196e-01, -6.56817760e-03,  1.64513215e-01,
         -2.76236683e-01,  4.93253887e-01,  1.61598444e-01,
          3.96686196e-01, -3.28653082e-02,  1.00235648e-01,
         -4.70762134e-01,  1.92561150e-02,  2.64999866e-01,
         -2.61691093e-01,  6.81557953e-01,  8.88397321e-02,
          2.50844598e-01, -2.23523945e-01, -1.75546139e-01,
         -2.83911645e-01,  3.04234624e-01,  4.32821453e-01,
         -1.83674291e-01,  2.34544322e-01,  6.35027885e-02,
          1.44358784e-01, -3.61819208e-01,  1.91147804e-01,
          3.36834610e-01,  1.94177255e-01,  2.07793072e-01,
          2.26775676e-01,  6.97420090e-02, -3.99475358e-02,
         -1.99529424e-01,  1.00800827e-01,  5.04345037e-02,
          3.17224711e-01,  3.17197651e-0

In [9]:




# Function for query processing
def process_query(user_input):
    tokens = tokenizer(user_input, return_tensors="pt")
    embedding = embedding_model(**tokens).last_hidden_state.mean(dim=1).detach().numpy()

    # Retrieve top-k similar entries
    _, indices = index.search(embedding, k=2)
    retrieved_restaurants = [restaurant_data[i] for i in indices[0]]

    # Generate response
    retrieved_texts = ", ".join([f"{r['name']} - {r['review']}" for r in retrieved_restaurants])
    input_ids = generation_tokenizer(f"User: {user_input}\nBot: Here are some options: {retrieved_texts}", return_tensors="pt").input_ids
    output = generation_model.generate(input_ids, max_length=150, num_return_sequences=1)
    response = generation_tokenizer.decode(output[0], skip_special_tokens=True)

    return response

# Example usage
if __name__ == '__main__':
    user_query = "I'm looking for a good Italian restaurant in San Francisco."
    response = process_query(user_query)
    print(response)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


User: I'm looking for a good Italian restaurant in San Francisco.
Bot: Here are some options: Tony's Pizza Napoletana - Great pizzas!, Flour + Water - Delicious handmade pasta.
Bot: I'm looking for a good Italian restaurant in San Francisco.
Bot: Here are some options: Tony's Pizza Napoletana - Great pizzas!, Flour + Water - Delicious handmade pasta.
Bot: I'm looking for a good Italian restaurant in San Francisco.
Bot: Here are some options: Tony's Pizza Napoletana - Great pizzas!, Flour + Water - Delicious handmade pasta.
Bot: I'm looking for a good Italian restaurant in San Francisco.
Bot: Here
